## 引入套件

In [ ]:
import re
import os
import numpy as np
import nltk
import string
from pprint import pprint
try:
    import wget
except:
    !pip install wget
import wget
try:
    import gdown
except:
    !pip install gdown
import gdown
try:
    import jieba.posseg as pseg
except:
    !pip install jieba
import jieba.posseg as pseg
import matplotlib.pyplot as plt
import pandas as pd
from pylab import mpl
try:
    import transformers
except:
    !pip install transformers==4.11.3
try:
    import ckip_transformers
except:
    !pip install ckip-transformers==0.2.7
from ckip_transformers.nlp import CkipWordSegmenter
from ckip_transformers.nlp import CkipPosTagger
from ckip_transformers.nlp import CkipNerChunker

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 讀取資料

In [ ]:
product_data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/cat_train_v2.csv")
product_data=product_data[["name","product"]]
brand_data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/brand_train.csv")
brand_data=brand_data[["name","brand"]]

## 資料集分析

In [ ]:
#把資料都完整列出來
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)
count = product_data['product'].value_counts()

In [ ]:
count

In [ ]:
#將目標的單一分類提出來看是否為217
product_type=[x for x in set(product_data["product"])]
clean_product_type=[]
for x in product_type:
  temp=x.split(";")
  for x2 in temp:
    clean_product_type.append(x2)
clean_product_type=set(clean_product_type)
len(clean_product_type)

217

In [ ]:
#將目標的單一分類提出來看是否為217
product_type=[x for x in set(product_data["product"])]
product_class={}
for x in product_type:
  temp=x.split(";")
  for x2 in temp:
    product_class.update({x2:0})
len(product_class)

217

In [ ]:
#將品類的負數種類切割
for x in range(len(product_data)):
  product=product_data.iloc[x]['product']
  product=product.replace("/",";").split(";")
  product_data.iloc[x]['product']=product
product_data

In [ ]:
#品類關鍵字有出現在品名的有多少
special_data=pd.DataFrame(columns=["name","product"])
for x in range(len(product_data)):
  name=product_data.iloc[x]['name']
  product=product_data.iloc[x]['product']
  for x2 in product:
    if x2 in name:
      special_data=special_data.append({"name":name,"product":product},ignore_index=True)
      break
print(len(special_data))
print(len(special_data)/len(product_data))

44697
0.4842263774835872


In [ ]:
#品牌關鍵字沒有出現在品名的有多少


## 資料預處理

In [ ]:
#資料太多我先抽一部分做
sample=product_data.sample(frac=0.01,replace=False,random_state=1,ignore_index=True)

In [ ]:
#全小寫
for x in sample.keys():
  sample[x]=sample[x].str.lower()

In [ ]:
#去除停用字、符號
PUNCT_TO_REMOVE = string.punctuation
PUNCT_TO_REMOVE=PUNCT_TO_REMOVE+"【】"
sample['name']=sample['name'].str.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [ ]:
#去除emoji
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
sample['clean_name']=0
index=0
for x in sample["name"]:
  sample["clean_name"][index]=remove_emoji(x) #不知為何不用新欄位存就無法更新進去
  index+=1
sample.drop(['name'],axis=1,inplace=True)
sample=sample[['clean_name','product']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## NER流程

In [ ]:
print('Initializing drivers ... NER')
CKIP_NER = CkipNerChunker(level=3, device=0)

Initializing drivers ... NER


Downloading:   0%|          | 0.00/3.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/388M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [ ]:
sample["NER"]=0
index=0
for x in sample["clean_name"]:
  sample["NER"][index]=CKIP_NER(x)
  index+=1

Inference: 100%|██████████| 1/1 [00:00<00:00,  6.91it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:881: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If y

In [ ]:
sample.head(10)

,clean_name,product,NER
0,逸品日式杯麵東京醬油風味康是美,速食麵/泡麵,"[[(逸, PERSON, (0, 1))], [], [(日, GPE, (0, 1))]..."
1,new balance 童 wide 經典復古鞋,運動鞋,"[[], [], [], [], [], [], [], [], [], [], [], [..."
2,天地合補advanced金盞花葉黃素濃縮飲15ml×32入,葉黃素,"[[], [], [], [], [], [], [], [], [], [], [], [..."
3,t綠茶植萃淨化沐浴露,沐浴乳,"[[], [], [], [], [], [], [], [(沐, PERSON, (0, ..."
4,媚比琳fit me反孔特霧粉底液 115 30ml,粉底液,"[[], [], [(琳, PERSON, (0, 1))], [], [], [], []..."
5,汪喵狗狗主食罐80g鹿肉,狗乾糧罐頭,"[[(汪, PERSON, (0, 1))], [], [], [], [], [], []..."
6,毛寶天然植萃pm25洗衣精2000gx6入,洗衣精,"[[(毛, PERSON, (0, 1))], [], [], [], [], [], []..."
7,艾惟諾燕麥益敏修護保濕霜200ml,面霜/乳霜,"[[], [], [], [(燕, PERSON, (0, 1))], [], [], []..."
8,味丹美味小鋪牛肉麵72g5入,速食麵/泡麵,"[[], [], [(美, GPE, (0, 1))], [], [], [], [], [..."
9,hills 希爾思成犬 7歲以上雞肉、大麥與糙米配方 15lb68kg603797,狗乾糧罐頭,"[[], [], [], [], [], [], [], [(爾, PERSON, (0, ..."


## 分群流程

In [ ]:
CKIP_WS = CkipWordSegmenter(level=1, device=0)

Downloading:   0%|          | 0.00/832 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/15.2M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [ ]:
sample_cluster=sample[["clean_name","product"]]
sample_cluster["ws"]=0
index=0
############ <- to be fixed 
for x in sample_cluster["clean_name"]:
  sample_cluster["ws"][index]=CKIP_WS(x)
  index+=1
sample_cluster.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
Inference: 100%|██████████| 1/1 [00:00<00:00, 58.94it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

,clean_name,product,ws
0,逸品日式杯麵東京醬油風味康是美,速食麵/泡麵,"[[逸], [品], [日], [式], [杯], [麵], [東], [京], [醬], ..."
1,new balance 童 wide 經典復古鞋,運動鞋,"[[n], [e], [w], [ ], [b], [a], [l], [a], [n], ..."
2,天地合補advanced金盞花葉黃素濃縮飲15ml×32入,葉黃素,"[[天], [地], [合], [補], [a], [d], [v], [a], [n], ..."
3,t綠茶植萃淨化沐浴露,沐浴乳,"[[t], [綠], [茶], [植], [萃], [淨], [化], [沐], [浴], ..."
4,媚比琳fit me反孔特霧粉底液 115 30ml,粉底液,"[[媚], [比], [琳], [f], [i], [t], [ ], [m], [e], ..."


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
sample_cluster['LABEL'] = LabelEncoder().fit_transform(sample_cluster['product'])

In [ ]:
sample_cluster.head()

,clean_name,product,ws,w_vector,LABEL
0,逸品日式杯麵東京醬油風味康是美,速食麵/泡麵,"[[逸], [品], [日], [式], [杯], [麵], [東], [京], [醬], ...","[[-0.0053622723, 0.0023643016, 0.051033497, 0....",136
1,new balance 童 wide 經典復古鞋,運動鞋,"[[n], [e], [w], [ ], [b], [a], [l], [a], [n], ...","[[-0.0053622723, 0.0023643016, 0.051033497, 0....",138
2,天地合補advanced金盞花葉黃素濃縮飲15ml×32入,葉黃素,"[[天], [地], [合], [補], [a], [d], [v], [a], [n], ...","[[-0.0053622723, 0.0023643016, 0.051033497, 0....",120
3,t綠茶植萃淨化沐浴露,沐浴乳,"[[t], [綠], [茶], [植], [萃], [淨], [化], [沐], [浴], ...","[[-0.0053622723, 0.0023643016, 0.051033497, 0....",63
4,媚比琳fit me反孔特霧粉底液 115 30ml,粉底液,"[[媚], [比], [琳], [f], [i], [t], [ ], [m], [e], ...","[[-0.0053622723, 0.0023643016, 0.051033497, 0....",108


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    sample_cluster['clean_name'], sample_cluster['LABEL'], test_size=0.33, random_state=42)

In [ ]:
def show_result(predicted, predicted_proba, target):
    print('*'*50)
    print('predicted class of first 3 test data')
    print(predicted[:3])

    print('*'*50)
    print('predicted class proba. of first 3 test data')
    print(predicted_proba[:3])

    np.mean(predicted == target)
    print('*'*50)
    print('accuracy performance on test data')
    print(np.mean(predicted == target))

In [ ]:
tfidf_vect = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vect.fit_transform(X_train)
X_test_tfidf = tfidf_vect.transform(X_test)

In [ ]:
KNeighborsClassifier_clf = KNeighborsClassifier(n_neighbors=5, weights='distance')
print('*'*50)
print('KNeighbors classifier with TfidfVectorizer')
print(KNeighborsClassifier_clf)

# fit train data
KNeighborsClassifier_clf.fit(X_train_tfidf, y_train)

# predict the class and class proba.
predicted = KNeighborsClassifier_clf.predict(X_test_tfidf)
predicted_proba = KNeighborsClassifier_clf.predict_proba(X_test_tfidf)

show_result(predicted, predicted_proba, y_test)

**************************************************
KNeighbors classifier with TfidfVectorizer
KNeighborsClassifier(weights='distance')
**************************************************
predicted class of first 3 test data
[96 96 96]
**************************************************
predicted class proba. of first 3 test data
[[0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.2 0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.
  0.  0.  0.2 0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.

In [ ]:
from sklearn.cluster import KMeans
tfidf_vect = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf_vect.fit_transform(sample['clean_name'])

km=KMeans(n_clusters=217,
         init="k-means++",
         n_init=10,
         max_iter=300,
         tol=1e-04,
         random_state=0)
y_km = km.fit_predict(X_tfidf)
sample['cluster']=y_km

In [ ]:
sample_type=[x for x in set(sample["product"])]
clean_sample_type=[]
for x in sample_type:
  temp=x.split(";")
  for x2 in temp:
    clean_sample_type.append(x2)
clean_sample_type=set(sample_type)
clean_sample_type=list(clean_sample_type)
len(clean_sample_type)

161

In [ ]:
from scipy import stats
correct=0
for x in range(len(clean_sample_type)):
  temp=list(sample[sample["product"]==clean_sample_type[x]]['cluster'])
  mode = stats.mode(temp)
  correct+=temp.count(mode[0])
print(correct/len(sample))

0.7475622968580715
